# Sensitive Model Search

by adjusting `o_random_seed` and `parameter_generation_seed`

## Init

In [1]:
import os

path = os.getcwd()
# find the string 'project' in the path, return index
index_project = path.find('project')
# slice the path from the index of 'project' to the end
project_path = path[:index_project+7]
# set the working directory
os.chdir(project_path+'/src')
print(f'Project path set to: {os.getcwd()}')

Project path set to: c:\Github\new-peak-project\src


In [2]:
from dotenv import dotenv_values
config = dotenv_values(".env")
print(config["DATA_PATH"])

I:\My Drive\DAWSON PHD PROJECT\Biomarker Data Repository\data\new-peak-project\experiments


In [3]:
from models.ModelBuilder import ModelBuilder
from models.Reaction import Reaction
from models.ReactionArchtype import ReactionArchtype
from models.ArchtypeCollections import *
from models.Utils import *

import matplotlib.pyplot as plt
import seaborn as sns
import roadrunner
import numpy as np
import pandas as pd

# import scikit-learn
from sklearn.linear_model import LinearRegression
# tree models and support vector machines
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# import pearson correlation
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from copy import deepcopy

## Analysis

In [4]:
import os 

### parameters 
notebook_name = 'exp4_drug_model_search'
sub_id = '2'

## Model parameters
no_observable_species = 15
no_feedback_regulations = 10
specie_value_range = (5, 5000)
param_range = (0.05, 20)
param_multiplier_range = (0.5, 1.5)
model_name = 'sensitive_model_search'

drug_name = 'D0'
drug_conc = 5000
drug_time = 500

## Sensitivity analysis parameters 
o_random_seeds = list(range(1,10))
parameter_random_seeds = list(range(1, 10))
species_perturbation_range = np.arange(1, 5000, 500)

## Simulation parameters 
simulation_time = 1000 
simulation_step = 100

## General parameters
parallelise = True
save_figures = True 
experiment_id = notebook_name + '_' + sub_id
experiment_folder = config['DATA_PATH'] + '/' + experiment_id + '/'
if not os.path.exists(experiment_folder):
    os.makedirs(experiment_folder)
    
print(experiment_folder)

I:\My Drive\DAWSON PHD PROJECT\Biomarker Data Repository\data\new-peak-project\experiments/exp4_drug_model_search_2/


In [5]:
## Helper functions
import warnings

from models.SensitivityAnalysis import sensitivity_analysis, get_sensitivity_score, extract_states_from_results

In [6]:
results = []
# use joblib to parallelise the code
from joblib import Parallel, delayed
from models.Solver.RoadrunnerSolver import RoadrunnerSolver
from models.DrugModelSpecification import DrugModelSpecification, Drug


import numpy as np

def fitness_function(values):
    values = np.array(values)

    # Penalty: any value > 50
    penalty_high = np.sum(np.maximum(values - 50, 0)) * 10

    # Penalty: mean deviates from 20
    mean_penalty = abs(np.mean(values) - 20) * 0.5

    # Reward: higher standard deviation
    std_reward = np.std(values) * 3

    # Penalty: too many values < 5
    small_count = np.sum(values < 5)
    total_count = len(values)
    if small_count > total_count / 2:
        small_penalty = (small_count - total_count / 2) * 15  # heavy penalty per extra small value
    else:
        small_penalty = 0

    # Total penalty (lower is better)
    total_score = penalty_high + mean_penalty + small_penalty - std_reward
    return total_score


def run_sensitivity_analysis(o_random_seed, parameter_random_seed, verbose=0):
    # Generate the model 
    model_spec = DrugModelSpecification()
    model_spec.generate_specifications(o_random_seed, no_observable_species, no_feedback_regulations, verbose=0)
    drug_0 = Drug(drug_name, drug_time, drug_conc)
    np.random.seed(o_random_seed)
    # add random 'up' and 'down' regulations to the drug
    regulation_dir = []
    for i, s in enumerate(model_spec.A_species):
        regulation_dir.append(np.random.choice(['up', 'down']))
        drug_0.add_regulation(s, 'up')
    model_spec.add_drug(drug_0)
    G0 = model_spec.generate_network(model_name, specie_value_range, param_range, param_multiplier_range, random_seed=parameter_random_seed, verbose=0)
    solver = RoadrunnerSolver()
    solver.compile(G0.get_sbml_model())
    all_states = []
    for i in range(no_observable_species):
        all_states.append('A'+str(i))
    for i in range(no_observable_species):
        all_states.append('B'+str(i))
                
    all_init_species_results = []
    for init_species in all_states: 
        all_results = sensitivity_analysis(G0, solver, init_species, species_perturbation_range, simulation_time, simulation_step)
        all_init_species_results.append(all_results)

    # extract the last time point of Cp for each init species
    Cp_final_states = []
    for init_species in all_init_species_results: 
        Cp_final_states.append(extract_states_from_results(init_species, 'Cp', -1))
        
    state_sensitivity = get_sensitivity_score(Cp_final_states)
    # get a balance of sensitiv
    mean = np.mean(state_sensitivity)
    std_dev = np.std(state_sensitivity)
    cv = std_dev / mean
    # get the mean of the sensitivity score
    sens_score = np.mean(state_sensitivity)
    median = np.median(state_sensitivity)
    # use a custom fitness function to punish large values in `state_sensitivity`
    # for each value in `state_sensitivity`, ideal target is a tailed distribution with mean around 20 and no value above 50 
    fitness = fitness_function(state_sensitivity)
    all_species = model_spec.A_species + model_spec.B_species
    if verbose: 
        print(f'Random seed: {o_random_seed}, Parameter random seed: {parameter_random_seed}, Fitness: {fitness} Mean: {sens_score}, CV: {cv}, Median: {median}')
        if verbose == 2:
            print('State sensitivity as a function of init species:')
            for i, s in enumerate(all_species): 
                print(f'Init species: {s}, Sensitivity score: {state_sensitivity[i]}')
    return [o_random_seed, parameter_random_seed, fitness, cv, sens_score, median]





In [7]:
res = run_sensitivity_analysis(6, 7, verbose=2)

Random seed: 6, Parameter random seed: 7, Fitness: 140.25138544622695 Mean: 5.378370473965739, CV: 2.0164630301942554, Median: 1.7462788114004368
State sensitivity as a function of init species:
Init species: A0, Sensitivity score: 18.97001739487439
Init species: A1, Sensitivity score: 0.523682455759257
Init species: A2, Sensitivity score: 4.866154593663467
Init species: A3, Sensitivity score: 0.09842966737605252
Init species: A4, Sensitivity score: 0.9366618215282898
Init species: A5, Sensitivity score: 0.022423603885158627
Init species: A6, Sensitivity score: 7.607626045852427
Init species: A7, Sensitivity score: 0.5010743156481681
Init species: A8, Sensitivity score: 0.24494805396274444
Init species: A9, Sensitivity score: 0.18235051786138
Init species: A10, Sensitivity score: 1.6133317983850333
Init species: A11, Sensitivity score: 2.913276591537155
Init species: A12, Sensitivity score: 1.8792258244158404
Init species: A13, Sensitivity score: 2.1058744863466927e-06
Init species: A1

In [8]:
from joblib import cpu_count
print(f'Joblib running on CPU cores: {cpu_count()}')

results = Parallel(n_jobs=-1)(delayed(run_sensitivity_analysis)(o_random_seed, parameter_random_seed) for o_random_seed in o_random_seeds for parameter_random_seed in parameter_random_seeds)
# convert to pandas dataframe
df = pd.DataFrame(results, columns=['o_random_seed', 'parameter_random_seed', 'fitness','cv', 'mean', 'median'])

# sort the dataframe by sensitivity score
df = df.sort_values(by='fitness', ascending=True)
df

Joblib running on CPU cores: 16


,o_random_seed,parameter_random_seed,fitness,cv,mean,median
51,6,7,140.251385,2.016463,5.378370,1.746279
60,7,7,142.919359,2.663963,3.777798,0.625471
70,8,8,149.492718,1.315832,2.362516,0.963188
59,7,6,163.073864,1.501026,2.383759,0.850233
14,2,6,166.986931,1.474385,1.627629,0.305365
...,...,...,...,...,...,...
73,9,2,602.775767,3.882309,4.589310,0.193272
72,9,1,642.337780,3.631694,5.321714,0.191237
74,9,3,739.536870,3.273050,6.044159,0.260582
79,9,8,740.769430,3.778990,5.388159,0.041826


## DE 

In [48]:
# Preparing variables for differential evolution
from models.DrugModelSpecification import DrugModelSpecification, Drug
from models.Solver.RoadrunnerSolver import RoadrunnerSolver
model_spec = DrugModelSpecification()

no_observable_species = 2
no_feedback_regulations = 1
drug_name = 'D0'
drug_conc = 5000
drug_time = 500
model_name = 'de_model'
specie_value_range = (5, 5000)
param_range = (0.05, 20)
param_multiplier_range = (0.5, 1.5)
simulation_time = 1000
simulation_step = 100
specie_value_range = (5, 5000)

model_spec.generate_specifications(5, no_observable_species, no_feedback_regulations, verbose=0)
drug_0 = Drug(drug_name, drug_time, drug_conc)
np.random.seed(5)
# add random 'up' and 'down ' regulations to the drug
regulation_dir = []
for i, s in enumerate(model_spec.A_species):
    regulation_dir.append(np.random.choice(['up', 'down']))
    drug_0.add_regulation(s, regulation_dir[-1])
model_spec.add_drug(drug_0)
G0 = model_spec.generate_network(model_name, specie_value_range, param_range, param_multiplier_range, random_seed=5, verbose=0)
solver = RoadrunnerSolver()
solver.compile(G0.get_sbml_model())
# state bounds
states = G0.get_state_variables()
del states['C']
del states['Cp']
num_states = len(states)    
state_bounds = [(specie_value_range[0], specie_value_range[1]) for _ in range(num_states)]
state_names = list(states.keys())
perturb_state_names = [val for val in state_names if 'p' not in val]
# parameter bounds
params = G0.get_parameters()
param_vals = list(params.values())
param_names = list(params.keys())
param_bounds = [(val*param_multiplier_range[0], val*param_multiplier_range[1]) for val in param_vals]

# combine state and parameter bounds
bounds = state_bounds + param_bounds
print(f'Number of states: {num_states}, Number of parameters: {len(param_bounds)}')
print(f'Number of bounds: {len(bounds)}')

Number of states: 8, Number of parameters: 27
Number of bounds: 35


In [49]:
from models.SensitivityAnalysis import extract_states_from_results, get_sensitivity_score


def fitness_function(values):
    values = np.array(values)

    # Penalty: any value > 50
    penalty_high = np.sum(np.maximum(values - 50, 0)) * 10

    # Penalty: mean deviates from 20
    mean_penalty = abs(np.mean(values) - 20) * 0.5

    # Reward: higher standard deviation
    std_reward = np.std(values) * 3

    # Penalty: too many values < 5
    small_count = np.sum(values < 5)
    total_count = len(values)
    if small_count > total_count / 2:
        small_penalty = (small_count - total_count / 2) * 15  # heavy penalty per extra small value
    else:
        small_penalty = 0

    # Total penalty (lower is better)
    total_score = penalty_high + mean_penalty + small_penalty - std_reward
    return total_score


# define the fitness function for differential evolution
def fitness_function_de(x, 
                        solver, 
                        state_names, 
                        param_names, 
                        perturb_state_names,
                        simulation_time,
                        simulation_step,
                        specie_value_range):
    
    all_init_species_results = []
    for init_specie in perturb_state_names: 
        all_results = []
        for specie in specie_value_range:
            # first set the state values as a whole from the x vector
            # print(f'Setting state values: {state_names} to {x[:num_states]}')
            states_values = {state_names[i]: np.float64(x[i]) for i in range(num_states)}
            solver.set_state_values(states_values)
            # then set the parameters values from the x vector
            params_values = {param_names[i]: np.float64(x[i+num_states]) for i in range(len(param_names))}
            solver.set_parameter_values(params_values)
            # finally set the state values for the specific specie, for perturbation
            solver.set_state_values({init_specie: np.float64(specie)})
            res = solver.simulate(0, simulation_time, simulation_step)
            all_results.append(res)
        all_init_species_results.append(all_results)

    # extract the last time point of Cp for each init species
    Cp_final_states = []
    for init_specie in all_init_species_results: 
        Cp_final_states.append(extract_states_from_results(init_specie, 'Cp', -1))
        
    state_sensitivity = get_sensitivity_score(Cp_final_states)
    fitness = fitness_function(state_sensitivity)
    # print(f'Fitness: {fitness}')
    return fitness



In [50]:
# test fitness function
x = np.random.uniform(0, 5000, len(state_bounds) + len(param_bounds))
fitness = fitness_function_de(x, solver, state_names, param_names, perturb_state_names, simulation_time, simulation_step, specie_value_range)
print(f'Fitness: {fitness}')

Fitness: 39.99164856279031


In [51]:
from scipy.optimize import differential_evolution
import time

progress_err = []
start_time = time.time()

def progress_bar(current, total, iteration, current_best, elapsed_time, bar_length=30):
    fraction = current / total
    if fraction > 1:
        fraction = 1

    arrow = int(fraction * bar_length - 1) * '-' + '>'
    padding = int(bar_length - len(arrow)) * ' '

    ending = '\n' if current == total else '\r'

    end_time = time.time()

    print(f'Cost: {current_best:.2f}| Progress: [{arrow}{padding}] {int(fraction*100)}%| Gen: {iteration}| Time per gen (s): {elapsed_time:.3f}', end=ending)

def callback(x, convergence):
    end_time = time.time()
    n = len(progress_err)
    elapsed_time = (end_time - start_time) / n if n > 0 else 0
    current_best = fitness_function_de(x, solver, state_names, param_names, perturb_state_names, simulation_time, simulation_step, specie_value_range)
    progress_err.append(current_best)
    progress_bar(convergence, 1, len(progress_err), current_best, elapsed_time)
    
# run the differential evolution algorithm
result = differential_evolution(fitness_function_de, 
                                bounds, 
                                maxiter=10, 
                                popsize=10, 
                                args=(solver, state_names, param_names, perturb_state_names, simulation_time, simulation_step, specie_value_range),
                                callback=callback,              
                                )

In [54]:
print(f'Best fitness: {result.fun}')
# reformat result.x to floats 
result_x = [float(x) for x in result.x]
print(f'Best parameters: {result_x}')

Best fitness: 39.76667727912817
Best parameters: [4732.708308405605, 492.4823113915213, 4128.359575904011, 722.7112866491364, 75.65698316336739, 1685.958338267275, 50.19032894991551, 310.9279187875759, 404.9432631353717, 30.8030375911892, 139.01720329678966, 3.9465308552462663, 2.293379352414888, 2.3698750857108437, 756.981089511704, 17.04055457374324, 634.9428664154642, 13.877254403352643, 0.3760497121432218, 265.3555695720396, 9.928246187230908, 617.3147051394207, 17.504701399476183, 1.6205423885743095, 1800.2881158556338, 106.81675193619051, 13.382456187091337, 10.013974970801934, 1.7639549205697564, 689.5968376402468, 2.2132293216691235, 1.205683419573305, 0.3427621095045505, 526.5870997872134, 6.047232444226003]


In [ ]:
from scipy.optimize import dual_annealing

start_time = time.time()
def anneal_callback(x, f, context):
    end_time = time.time()
    n = len(progress_err)
    elapsed_time = (end_time - start_time) / n if n > 0 else 0
    current_best = fitness_function_de(x, solver, state_names, param_names, perturb_state_names, simulation_time, simulation_step, specie_value_range)
    progress_err.append(current_best)
    progress_bar(f, 1, len(progress_err), current_best, elapsed_time)
    # print(f'Current best: {current_best}')

# run the dual annealing algorithm
res = dual_annealing(fitness_function_de, bounds, callback=anneal_callback, 
                     args=(solver, state_names, param_names, perturb_state_names, simulation_time, simulation_step, specie_value_range), 
                    )

In [57]:
print(f'Best fitness: {res.fun}')
print(f'Best parameters: {res.x}')

Best fitness: 39.89203998286316
Best parameters: [3.58800015e+03 1.62199659e+03 3.87452773e+03 3.18962608e+03
 3.69218866e+02 1.87147419e+02 3.27845901e+02 3.39381498e+03
 3.37395749e+02 3.43147656e+01 2.25915007e+02 9.94600539e+00
 3.20343797e+00 2.42482505e+00 5.50641797e+02 2.15028173e+01
 4.65814751e+02 8.33017683e+00 3.33855079e-01 3.03982205e+02
 2.01338811e+01 8.10291922e+02 1.82304325e+01 1.26991962e+00
 1.47978816e+03 6.68859529e+01 1.30878363e+01 9.95699742e+00
 9.05384361e-01 3.95709210e+02 4.17951400e+00 1.22475852e+00
 2.81863540e-01 8.87127762e+02 8.43114064e+00]
